In [0]:
%pip install schedule
%pip install lxml
%pip install openpyxl

Python interpreter will be restarted.
Collecting schedule
 Downloading schedule-1.2.1-py2.py3-none-any.whl (11 kB)
Installing collected packages: schedule
Successfully installed schedule-1.2.1
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting lxml
 Downloading lxml-5.1.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.0 MB)
Installing collected packages: lxml
Successfully installed lxml-5.1.0
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting openpyxl
 Downloading openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
Collecting et-xmlfile
 Downloading et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Installing collected packages: et-xmlfile, openpyxl
Successfully installed et-xmlfile-1.1.0 openpyxl-3.1.2
Python interpreter will be restarted.

In [0]:
import pandas as pd

# Para actualizar el archivo CSV diariamente se utiliza el módulo schedule de Python para programar la ejecución del script cada día.
import schedule
import time

from datetime import datetime

# Cumpleaños de Leo
leo_birthday = datetime(1987, 6, 24)

# Término de contratos
barcelona_exit = datetime(2021, 5, 30)
psg_exit = datetime(2023, 6, 30)


# Definir df como una variable global
df = None

def update_csv():
    global df  # Definir df como una variable global
    # Obtenemos la tabla de ejemplo de una página web
    url = "http://messi.starplayerstats.com/en/goals/0/0/all/0/0/0/t/all/all/0/0/1"
    # Para obtener la primera tabla
    goals_table = pd.read_html(url)[0]

    # Estableceremos el Dataframe donde guardaremos la tabla
    df = pd.DataFrame(goals_table)

    # Elimino la última columna (son las flechas que me llevan al detalle del partido)
    df = df.drop(columns=["Unnamed: 11"])

    # Renombro las columnas
    df = df.set_axis(
        [
            "N_of_Goal",
            "Date",
            "Competition",
            "Home_team",
            "Result",
            "Away_team",
            "Minute",
            "Partial_Score",
            "What",
            "How",
            "Jersey",
        ],
        axis=1,
    )
    df["N_of_Goal"] = df["N_of_Goal"].astype("int")
    df["Jersey"] = df["Jersey"].astype("int")
    # df["Date"] = df["Date"].astype("datetime64[us]")
    df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")


    # Dividir "Competition" en 2 columnas: una para la abreviación, y otra para el nombre de la competición. Esto me permite identificar los amistosos por su sigla, y todo lo demás como partidos oficiales
    def split_competition(val):
        return val[:3], val[4:]

    # Aplicar la función personalizada a la columna "Competition"
    df[["Competition_abb", "Competition_name"]] = (
        df["Competition"].apply(split_competition).apply(pd.Series)
    )
    # Eliminar la columna "column_name"
    df = df.drop("Competition", axis=1)


    # Dividir los goles del resultado en 2 columnas
    df[["Goals_H", "Goals_A"]] = df["Result"].str.split("-", expand=True)
    # Convertir los "result" en int
    df["Goals_H"] = df["Goals_H"].astype("int")
    df["Goals_A"] = df["Goals_A"].astype("int")
    # Eliminar la columna "column_name"
    df = df.drop("Result", axis=1)

    # Crear una nueva columna "Result" que contenga los valores "Home", "Away" o "Draw" dependiendo de los valores de "Goals_H" y "Goals_A"
    df["Result"] = df.apply(
        lambda number: "Home"
        if number["Goals_H"] > number["Goals_A"]
        else "Away"
        if number["Goals_H"] < number["Goals_A"]
        else "Draw",
        axis=1,
    )


    # Calcular la edad de Leo en cada gol
    from dateutil.relativedelta import relativedelta

    df["Leo_age"] = df["Date"].apply(
        lambda goal: relativedelta(goal.date(), leo_birthday.date()).years
    )


    # Identificar en qué equipo estaba jugando Leo cuando marcó el gol
    df["Leo_team"] = df.apply(
        lambda goal: "Argentina"
        if goal["Home_team"] == "Argentina"
        else "Argentina"
        if goal["Away_team"] == "Argentina"
        else "FC Barcelona"
        if goal["Date"] < barcelona_exit
        else "Paris Saint-Germain"
        if goal["Date"] < psg_exit
        else "Inter Miami CF",
        axis=1,
    )


    # Identificar a qué equipo le anotó el gol
    df["Scored_team"] = df.apply(
        lambda goal: goal["Away_team"]
        if goal["Home_team"] == goal["Leo_team"]
        else goal["Home_team"],
        axis=1,
    )


    # Identificar si estaba jugando de local o visitante
    df["Home_or_Away"] = df.apply(
        lambda goal: "Home" if goal["Home_team"] == goal["Leo_team"] else "Away", axis=1
    )


    # Identificar si Leo ganó, empató o perdió
    df["Leo_result"] = df.apply(
        lambda goal: "Tied"
        if goal["Result"] == "Draw"
        else "Won"
        if goal["Result"] == goal["Home_or_Away"]
        else "Lost",
        axis=1,
    )


    # Dividir los goles del resultado parcial en 2 columnas
    df[["Partial_Score_H", "Partial_Score_A"]] = df["Partial_Score"].str.split(
        "-", expand=True
    )
    # Convertir los "result" en int
    df["Partial_Score_H"] = df["Partial_Score_H"].astype("int")
    df["Partial_Score_A"] = df["Partial_Score_A"].astype("int")
    # Eliminar la columna "column_name"
    df = df.drop("Partial_Score", axis=1)


    # Agregar una nueva columna "Minute_adjusted" al DataFrame df
    df['Minute_adjusted'] = df['Minute']

    # Definir una función para ajustar los minutos en caso de tiempo adicional
    def adjust_minute(minute):
        if '+' in minute:
            # Dividir el tiempo adicional en dos partes y sumarlas
            parts = minute.split('+')
            adjusted_minute = sum(int(part) for part in parts)
            return adjusted_minute
        else:
            # Mantener el mismo valor si no hay tiempo adicional
            return int(minute)

    # Aplicar la función adjust_minute a la columna "Minute_adjusted"
    df['Minute_adjusted'] = df['Minute_adjusted'].apply(adjust_minute)

    # Convertir la columna "Goal_time_period" al tipo de datos texto
    df['Minute_adjusted'] = df['Minute_adjusted'].astype('int')


    # Agregar una nueva columna "Goal_time_period" al DataFrame df
    df['Goal_time_period'] = df['Minute']

    # Definir una función para determinar el período de tiempo del gol
    def determine_time_period(minute):
        if '+' in minute:
            return 'Additional time'
        elif int(minute) <= 45:
            return 'First half'
        elif int(minute) <= 90:
            return 'Second half'
        else:
            return 'Additional time'

    # Aplicar la función determine_time_period a la columna "Goal_time_period"
    df['Goal_time_period'] = df['Goal_time_period'].apply(determine_time_period)

    # Convertir la columna "Goal_time_period" al tipo de datos texto
    df['Goal_time_period'] = df['Goal_time_period'].astype('str')


    # Guardar el DataFrame en un archivo CSV utilizando la función df.to_csv(), y en un archivo XLSX utilizando la función df.to_excel().
    df.to_csv("messi_goals.csv", index=False)
    df.to_excel("messi_goals.xlsx", sheet_name="Goals")
    return df


# Actualizar df
df = update_csv()

print(df)

N_of_Goal Date ... Minute_adjusted Goal_time_period
0 821 2023-10-17 ... 42 First half
1 820 2023-10-17 ... 32 First half
2 819 2023-09-07 ... 78 Second half
3 818 2023-08-26 ... 89 Second half
4 817 2023-08-20 ... 23 First half
.. ... ... ... ... ...
816 5 2006-01-22 ... 81 Second half
817 4 2006-01-15 ... 51 Second half
818 3 2005-11-27 ... 52 Second half
819 2 2005-11-02 ... 34 First half
820 1 2005-05-01 ... 90 Second half

[821 rows x 22 columns]

In [0]:
"""
import os

# Ruta completa del archivo CSV
csv_file_path = "messi_goals.csv"

# Verificar si el archivo existe antes de intentar borrarlo
if os.path.exists(csv_file_path):
    # Borrar el archivo CSV
    os.remove(csv_file_path)
    print("Archivo CSV borrado correctamente.")
else:
    print("El archivo CSV no existe en el directorio actual.")
"""

Out[2]: '\nimport os\n\n# Ruta completa del archivo CSV\ncsv_file_path = "messi_goals.csv"\n\n# Verificar si el archivo existe antes de intentar borrarlo\nif os.path.exists(csv_file_path):\n # Borrar el archivo CSV\n os.remove(csv_file_path)\n print("Archivo CSV borrado correctamente.")\nelse:\n print("El archivo CSV no existe en el directorio actual.")\n'

In [0]:
"""
import os

# Ruta completa del archivo CSV
xlsx_file_path = "messi_goals.xlsx"

# Verificar si el archivo existe antes de intentar borrarlo
if os.path.exists(xlsx_file_path):
    # Borrar el archivo CSV
    os.remove(xlsx_file_path)
    print("Archivo XLSX borrado correctamente.")
else:
    print("El archivo XLSX no existe en el directorio actual.")
"""

Out[3]: '\nimport os\n\n# Ruta completa del archivo CSV\nxlsx_file_path = "messi_goals.xlsx"\n\n# Verificar si el archivo existe antes de intentar borrarlo\nif os.path.exists(xlsx_file_path):\n # Borrar el archivo CSV\n os.remove(xlsx_file_path)\n print("Archivo XLSX borrado correctamente.")\nelse:\n print("El archivo XLSX no existe en el directorio actual.")\n'

In [0]:
"""
import os

# Obtener el directorio actual
current_dir = os.getcwd()

# Listar los archivos en el directorio actual
files = os.listdir(current_dir)

# Filtrar los archivos CSV
csv_files = [file for file in files if file.endswith(".csv")]

# Imprimir la lista de archivos CSV
print("Archivos CSV en el directorio actual:")
for csv_file in csv_files:
    print(csv_file)

# Filtrar los archivos XLSX
xlsx_files = [file for file in files if file.endswith(".xlsx")]

# Imprimir la lista de archivos XLSX
print("Archivos XLSX en el directorio actual:")
for xlsx_file in xlsx_files:
    print(xlsx_file)

print(" ")
print("Directorio actual:")
print(current_dir)
"""

Out[4]: '\nimport os\n\n# Obtener el directorio actual\ncurrent_dir = os.getcwd()\n\n# Listar los archivos en el directorio actual\nfiles = os.listdir(current_dir)\n\n# Filtrar los archivos CSV\ncsv_files = [file for file in files if file.endswith(".csv")]\n\n# Imprimir la lista de archivos CSV\nprint("Archivos CSV en el directorio actual:")\nfor csv_file in csv_files:\n print(csv_file)\n\n# Filtrar los archivos XLSX\nxlsx_files = [file for file in files if file.endswith(".xlsx")]\n\n# Imprimir la lista de archivos XLSX\nprint("Archivos XLSX en el directorio actual:")\nfor xlsx_file in xlsx_files:\n print(xlsx_file)\n\nprint(" ")\nprint("Directorio actual:")\nprint(current_dir)\n'